# Ground state solvers

In [1]:
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.drivers import PySCFDriver

driver = PySCFDriver(
    atom="H 0 0 0; H 0 0 0.735",
    basis="sto3g",
    charge=0,
    spin=0,
    unit=DistanceUnit.ANGSTROM,
)

es_problem = driver.run()

In [6]:
es_problem.properties.angular_momentum

In [5]:
from qiskit.algorithms.minimum_eigensolvers import (
    VQE, AdaptVQE, NumPyMinimumEigensolver)
from qiskit.algorithms.optimizers import SLSQP
from qiskit.primitives import Estimator
from qiskit_nature.second_q.circuit.library import HartreeFock, UCCSD
from qiskit_nature.second_q.mappers import JordanWignerMapper
from qiskit_nature.second_q.algorithms import GroundStateEigensolver

import numpy as np

mapper = JordanWignerMapper()

In [7]:
GroundStateEigensolver?

Init signature:
GroundStateEigensolver(
    qubit_converter: 'QubitConverter',
    solver: 'MinimumEigensolver | MinimumEigensolverFactory',
) -> 'None'
Docstring:      Ground state computation using a minimum eigensolver.
Init docstring:
Args:
    qubit_converter: A class that converts second quantized operator to qubit operator
                     according to a mapper it is initialized with.
    solver: Minimum Eigensolver or MESFactory object, e.g. the VQEUCCSDFactory.
File:           ~/mambaforge/envs/qiskit-env/lib/python3.10/site-packages/qiskit_nature/second_q/algorithms/ground_state_solvers/ground_state_eigensolver.py
Type:           ABCMeta
Subclasses:     

In [6]:
solver = GroundStateEigensolver(
    mapper = mapper,
    NumPyMinimumEigensolver(),
)
result = solver.solve(problem)
print(result)

SyntaxError: positional argument follows keyword argument (1954823658.py, line 4)

In [ ]:

# mapper = QubitConverter(JordanWignerMapper())
mapper = JordanWignerMapper()

ansatz = UCCSD(
    es_problem.num_spatial_orbitals,
    es_problem.num_particles,
    mapper,
    initial_state=HartreeFock(
        es_problem.num_spatial_orbitals,
        es_problem.num_particles,
        mapper,
    ),
)

vqe_solver = VQE(Estimator(), ansatz, SLSQP())
vqe_solver.initial_point = np.zeros(ansatz.num_parameters)

adapt_vqe = AdaptVQE(vqe_solver)


In [ ]:


vqe_calc = GroundStateEigensolver(mapper, vqe_solver)
vqe_res = vqe_calc.solve(es_problem)

adap_calc = GroundStateEigensolver(mapper, adapt_vqe)
adap_res = adap_calc.solve(es_problem)

print(f"VQE total ground state energy = {vqe_res.total_energies[0]:.4f}")
print(f"ADAPT-VQE total ground state energy = {adap_res.total_energies[0]:.4f}")

VQE total ground state energy = -1.1373
ADAPT-VQE total ground state energy = -1.1373


In [ ]:
from pyscf import gto, scf, mcscf
from pyscf.mcscf import avas #AVAS method to construct mcscf active space

from qiskit_nature_pyscf import QiskitSolver

In [ ]:
#  Initialisation de la structure moléculaire
H2_mol = gto.M(atom="H 0 0 0; H 0 0 .735", basis="sto-3g")

# Calculs HF 
H2_h_f = scf.RHF(H2_mol).run()

# Calculs post-HF
norb, nelec, mo =avas.avas(H2_h_f,['H 1s','H 1s'])

H2_cas = mcscf.CASCI(H2_h_f, norb, nelec)

# Intégration de l'algorithme quantique
# mapper = JordanWignerMapper()
# optimizer = SPSA()

H2_cas.fcisolver = QiskitSolver(adap_calc)

H2_cas.run()

converged SCF energy = -1.116998996754
CASCI E = -1.1373060356956  E(CI) = -1.85727503014458


In [7]:
import qiskit.tools.jupyter

%qiskit_version_table